# 01 — Business Understanding (Kedro + CRISP-DM)

In [1]:
# --- Inicializar Kedro sin magics ---
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project

# localizar la raíz del proyecto subiendo hasta encontrar pyproject.toml
project_path = Path.cwd()
while project_path != project_path.parent and not (project_path / "pyproject.toml").exists():
    project_path = project_path.parent
assert (project_path / "pyproject.toml").exists(), f"No encuentro pyproject.toml desde {Path.cwd()}"

bootstrap_project(str(project_path))
session = KedroSession.create(project_path=str(project_path))
ctx = session.load_context()
catalog = ctx.catalog

print("Proyecto:", project_path)

[09/15/25 22:08:55] INFO     Using                                                                  ]8;id=238873;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=711086;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\framework                
                             \project\rich_logging.yml' as logging configuration.                                  

[09/15/25 22:08:57] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=409071;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=554914;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

Proyecto: c:\Users\lttlk\Documents\Nueo\machinegame


In [2]:
# Diagnóstico rápido
import sys
try:
    import kedro
    print("Python:", sys.executable)
    print("Kedro:", kedro.__version__)
    print("Datasets (primeros 15):", list(catalog.list())[:15])
except Exception as e:
    print("No se pudo inspeccionar kedro:", e)

Python: c:\Users\lttlk\Documents\Nueo\.venv\Scripts\python.exe
Kedro: 1.0.0
No se pudo inspeccionar kedro: 'DataCatalogWithCatalogCommandsMixin' object has no attribute 'list'


In [3]:
# Helpers: cargar o reconstruir y reparación de encoding
from pathlib import Path
import pandas as pd

def load_or_build(name: str, pipe: str):
    """Intenta catalog.load(name); si falla, ejecuta el pipeline y vuelve a cargar."""
    try:
        return catalog.load(name)
    except Exception as e:
        print(f"[load_or_build] {name} no disponible o ilegible ({e}). Corriendo pipeline: {pipe} …")
        session.run(pipeline_name=pipe)
        return catalog.load(name)

def fix_reporting_csv_encoding(report_dir: Path | str = None, encoding_from: str = "latin1", encoding_to: str = "utf-8"):
    """Repara CSVs en 08_reporting que estén en ANSI/latin-1: reescribe en UTF-8.
    Si ya son UTF-8 válidos, los deja intactos.
    """
    if report_dir is None:
        report_dir = project_path / "data" / "08_reporting"
    report_dir = Path(report_dir)
    if not report_dir.exists():
        print("[fix_reporting_csv_encoding] No existe la carpeta:", report_dir)
        return

    fixed, skipped = [], []
    for p in report_dir.glob("*.csv"):
        try:
            _ = pd.read_csv(p, encoding=encoding_to, nrows=5)
            skipped.append(p.name)
        except Exception:
            try:
                df = pd.read_csv(p, encoding=encoding_from)
                df.to_csv(p, index=False, encoding=encoding_to)
                fixed.append(p.name)
            except Exception as e:
                print(f"[fix_reporting_csv_encoding] No se pudo reparar {p.name}: {e}")
    print("[fix_reporting_csv_encoding] Reparados (latin-1 -> utf-8):", fixed)
    print("[fix_reporting_csv_encoding] Ya estaban en utf-8:", skipped)

### Ejecutar pipeline de Business Understanding

In [4]:
session.run(pipeline_name="business_understanding")

[09/15/25 22:09:08] INFO     Kedro project machinegame                                               ]8;id=733760;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\framework\session\session.py\session.py]8;;\:]8;id=104042;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\framework\session\session.py#330\330]8;;\

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=297438;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=167703;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/15/25 22:09:09] WARNING  Workflow tracking is disabled during partial pipeline runs (executed  ]8;id=436919;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_viz\integrations\kedro\run_hooks.py\run_hooks.py]8;;\:]8;id=819332;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro_viz\integrations\kedro\run_hooks.py#135\135]8;;\
                             using --from-nodes, --to-nodes, --tags, --pipeline, and more).                        
                             `.viz/kedro_pipeline_events.json` will be created only during a full                  
                             kedro run. See issue                                                                  
                             https://github.com/kedro-org/kedro-viz/issues/2443 for more details.                  

[09/15/25 22:09:10] INFO     Using synchronous mode for loading and saving data. Use the    ]8;id=995348;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\sequential_runner.py\sequential_runner.py]8;;\:]8;id=167317;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\sequential_runner.py#59\59]8;;\
                             --async flag for potential performance gains.                                         
                             https://docs.kedro.org/en/stable/nodes_and_pipelines/run_a_pip                        
                             eline.html#load-and-save-asynchronously                                               

                    INFO     Loading data from games_raw (CSVDataset)...                       ]8;id=470316;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=189716;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

...

                    INFO     Saving data to vg_sales_describe (CSVDataset)...                  ]8;id=534403;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=730971;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py#1006\1006]8;;\

                    INFO     Completed node: vg_sales_describe_node                                   ]8;id=461424;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py\runner.py]8;;\:]8;id=686345;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py#245\245]8;;\

                    INFO     Completed 4 out of 4 tasks                                               ]8;id=270794;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py\runner.py]8;;\:]8;id=880035;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py#246\246]8;;\

                    INFO     Pipeline execution completed successfully in 0.9 sec.                    ]8;id=141281;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py\runner.py]8;;\:]8;id=629357;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\runner\runner.py#119\119]8;;\


{
    'eda_summary': kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/lttlk/Documents/Nueo/machinegame/data/08_reporting/eda_summary.csv'), protocol='file', load_args={'encoding': 'utf-8'}, save_args={'index': False, 'encoding': 'utf-8'}),
    'vg_sales_describe': kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/lttlk/Documents/Nueo/machinegame/data/08_reporting/vg_sales_describe.csv'), protocol='file', load_args={'encoding': 'utf-8'}, save_args={'index': False, 'encoding': 'utf-8'}),
    'steam_describe': kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/lttlk/Documents/Nueo/machinegame/data/08_reporting/steam_describe.csv'), protocol='file', load_args={'encoding': 'utf-8'}, save_args={'index': False, 'encoding': 'utf-8'}),
    'games_describe': kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/lttlk/Documents/Nueo/machinegame/data/08_reporting/games_describe.csv'), proto

### Reparar encoding de reporting (si fuese necesario)

In [5]:
fix_reporting_csv_encoding()

[fix_reporting_csv_encoding] Reparados (latin-1 -> utf-8): ['business_understanding.csv']
[fix_reporting_csv_encoding] Ya estaban en utf-8: ['dataset_inventory.csv', 'eda_summary.csv', 'games_describe.csv', 'steam_describe.csv', 'vg_sales_describe.csv']


### Cargar artefactos: `business_report` y `dataset_inventory`

In [6]:
business_report = load_or_build("business_report", "business_understanding")
dataset_inventory = load_or_build("dataset_inventory", "business_understanding")
display(business_report.head(30))
display(dataset_inventory)
print("business_report shape:", business_report.shape)

[09/15/25 22:09:29] INFO     Loading data from business_report (CSVDataset)...                 ]8;id=997869;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=817702;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

                    INFO     Loading data from dataset_inventory (CSVDataset)...               ]8;id=935566;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=318321;file://c:\Users\lttlk\Documents\Nueo\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

,seccion,item
0,Objetivos,Construir un dataset limpio y documentado a pa...
1,Objetivos,Identificar variables potencialmente útiles pa...
2,Preguntas de Negocio,¿Qué géneros y plataformas muestran mayor trac...
3,Preguntas de Negocio,¿Cómo se distribuyen las horas jugadas/accione...
4,Preguntas de Negocio,¿Qué variables podrían explicar las ventas por...
5,Métricas de Éxito,Datasets limpios en data/03_primary/
6,Métricas de Éxito,Reportes EDA en data/08_reporting/
7,Métricas de Éxito,Notebook por fase (1 a 3)


,dataset,filepath,exists,size_bytes,sample_columns
0,games_raw,data\01_raw\Games.csv,True,616319,"Console,GameName,Review,Score"
1,steam_raw,data\01_raw\steam-200k.csv,True,8958107,"151603712,The Elder Scrolls V Skyrim,purchase,..."
2,vg_sales_raw,data\01_raw\Video_Games_Sales_as_at_22_Dec_201...,True,1610764,"Name,Platform,Year_of_Release,Genre,Publisher,..."


business_report shape: (8, 2)


#### Conclusiones/Decisiones de negocio (rellena aquí)